In [218]:
%pip install huggingface-cli

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli
Note: you may need to restart the kernel to use updated packages.


Test de funcionamiento Neo4j para analisis.

Necesario:
    - Docker Neo4j arrancado
    - Activado entorno venv

Version 1.0

In [6]:
from flask import Flask, request, jsonify, render_template
import os
import json
from langchain import PromptTemplate, LLMChain
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader

In [52]:
# Initialize LLM and other components as in the original code
#local_llm = "neural-chat-7b-v3-1.Q4_K_M.gguf"
local_llm = "openhermes-2.5-mistral-7b.Q3_K_M.gguf"


In [53]:
# Parametros Modelos
config = {
          'max_new_tokens': 512,
          'repetition_penalty': 1.1,
          'temperature': 0,
          'context_length': 1024,
          'stream': False
          }

In [54]:
llm = CTransformers(
    model=local_llm,
    model_type="mistral",
    lib="avx2",
    **config
)

print("LLM Initialized....")

LLM Initialized....


In [9]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [10]:
# Modelo de embeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [11]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph

#url="bolt://localhost:7687"

In [12]:
# Conexión a la base de datos Neo4j
url = "neo4j://localhost:7687"
username = 'neo4j'
password = 'PROPILENO24a'

index_name = "NewEMBEDDING"  # default index name

In [13]:
# Función para inicializar el embedding para un tipo de nodo
def init_node_embedding(node_label, text_properties):
    return Neo4jVector.from_existing_graph(
        embedding=embeddings,
        url=url,
        username=username,
        password=password,
        index_name=f"{node_label.lower()}_index",
        node_label=node_label,
        text_node_properties=text_properties,
        embedding_node_property="NEWembedding"
    )

In [ ]:
print("Conexion Neo4j OK")

1.- Dada una pregunta query calcular su embedding
2.- Para cada nodo del grafo generar una propiedad EMBquery que contenga el valor de la similiraridad entre el vector de embedding del nodo y la pregunta

In [14]:
from langchain.vectorstores.neo4j_vector import Neo4jVector

In [15]:

node_label = "Question"
text_properties = ["string_es"]
k=3

In [16]:

existing_graph = Neo4jVector.from_existing_graph(
        embedding=embeddings,
        url=url,
        username=username,
        password=password,
        index_name=f"{node_label.lower()}_index",
        node_label=node_label,
        text_node_properties=text_properties,
        embedding_node_property="NEWembedding",
    )

In [17]:
# Embbeding pregunta
query = "Me duelen los pechos. ¿Que puedo hacer?"




In [13]:
# NO USAR
# TEST PARA VER SI FUNCIONA

from sentence_transformers import SentenceTransformer, util
import torch

# Modelo de embedding
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = SentenceTransformer(model_name, **model_kwargs)

# Texto de referencia
reference_text = "Este es el texto de referencia."

# Texto que deseas comparar
input_text = query

# Codificar los textos en embeddings
reference_embedding = embeddings.encode(reference_text, **encode_kwargs)
input_embedding = embeddings.encode(input_text, **encode_kwargs)

# Calcular la similitud del coseno entre los embeddings
cosine_similarity = util.pytorch_cos_sim(reference_embedding, input_embedding)

# El valor de cosine_similarity es un tensor de PyTorch que contiene la similitud del coseno
# Puedes obtener el valor como un número flotante de la siguiente manera:
cosine_similarity_value = cosine_similarity.item()

print(f"Similitud del coseno entre los textos: {cosine_similarity_value}")

Similitud del coseno entre los textos: 0.830841064453125


2.- Calculo de la similaridad del coseno entre la pregunta y todos los valores del grafo

In [78]:
# NO USAR

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from neo4j import GraphDatabase

# Modelo de embedding
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = SentenceTransformer(model_name, **model_kwargs)

# Función para calcular la similitud del coseno entre el query y las propiedades de los nodos
def calculate_similarity(tx, input_query):
    # Obtener los embeddings del query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)[0]
    
    result = tx.run("""
    MATCH (n)
    RETURN n.string_es AS node_text
    """)
    
    for record in result:
        node_text = record["node_text"]
        
        # Obtener el embedding del nodo y calcular la similitud del coseno
        node_embedding = embeddings.encode(node_text, **encode_kwargs)[0]
        similarity_score = cosine_similarity([query_embedding], [node_embedding])[0][0]
        
        # Actualizar el nodo con la similitud del coseno
        tx.run("""
        MATCH (n {string_es: $node_text})
        SET n.similaridadquery = $score
        """, node_text=node_text, score=similarity_score)

# Query que deseas comparar con las propiedades de los nodos
query = "Me duelen los pechos. ¿Que puedo hacer?"

# Configuración de la conexión a la base de datos Neo4j

# Conexión a la base de datos Neo4j
with GraphDatabase.driver(url, auth=(username, password)) as driver:
    with driver.session() as session:
        session.write_transaction(calculate_similarity, query)

print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

/tmp/ipykernel_23979/818607053.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(calculate_similarity, query)


ValueError: Expected 2D array, got 1D array instead:
array=[-0.01786431].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [13]:
# NO USAR

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from neo4j import GraphDatabase

# Configuración de la conexión a la base de datos Neo4j


# Modelo de embedding
model_name = "BAAI/bge-large-en"

# Conexión a la base de datos Neo4j
def calculate_similarity(tx, input_query):
    # Obtener los embeddings del query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)
    
    result = tx.run("""
    MATCH (n)
    RETURN n.string_es AS node_text
    """)
    
    for record in result:
        node_text = record["node_text"]
        
        # Obtener el embedding del nodo
        node_embedding = embeddings.encode(node_text, **encode_kwargs)
        
        # Calcular la similitud del coseno entre el query y el nodo
        similarity_score = cosine_similarity([query_embedding], [node_embedding])[0][0]
        
        # Actualizar el nodo con la similitud del coseno calculada
        tx.run("""
        MATCH (n {string_es: $node_text})
        SET n.similaridadquery = $score
        """, node_text=node_text, score=similarity_score)

# Query que deseas comparar con las propiedades de los nodos
query = "Me duelen los pechos. ¿Qué puedo hacer?"

# Configuración de los embeddings
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
with GraphDatabase.driver(url, auth=(username, password)) as driver:
    embeddings = SentenceTransformer(model_name, **model_kwargs)
    
    with driver.session() as session:
        #session.write_transaction(calculate_similarity, query)
        session.execute_write(calculate_similarity, query)

print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

In [85]:
# NO USAR
# MODIFICACIONES PARA ACELERAR EL PROCESO
# Tiempo previo: 26 min 43 seg

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from neo4j import GraphDatabase

# Configuración de la conexión a la base de datos Neo4j

# Modelo de embedding
model_name = "BAAI/bge-large-en"

# Conexión a la base de datos Neo4j
def calculate_similarity(tx, input_query):
    # Obtener los embeddings del query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    result = tx.run("""
    MATCH (n)
    RETURN n.string_es AS node_text
    """)

    queries_to_update = []

    for record in result:
        node_text = record["node_text"]

        # Obtener el embedding del nodo
        node_embedding = embeddings.encode(node_text, **encode_kwargs)

        # Calcular la similitud del coseno entre el query y el nodo
        similarity_score = cosine_similarity([query_embedding], [node_embedding])[0][0]

        # Preparar la consulta para actualizar el nodo
        queries_to_update.append({
            "node_text": node_text,
            "score": similarity_score
        })

    # Actualizar todos los nodos en una sola transacción
    for query_data in queries_to_update:
        tx.run("""
        MATCH (n {string_es: $node_text})
        SET n.similaridadquery = $score
        """, node_text=query_data["node_text"], score=query_data["score"])

# Query que deseas comparar con las propiedades de los nodos
query = "Me duelen los pechos. ¿Qué puedo hacer?"

# Configuración de los embeddings
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
with GraphDatabase.driver(url, auth=(username, password)) as driver:
    embeddings = SentenceTransformer(model_name, **model_kwargs)

    with driver.session() as session:
        session.write_transaction(calculate_similarity, query)

print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

/tmp/ipykernel_23979/1573953181.py:59: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(calculate_similarity, query)


Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Me duelen los pechos. ¿Qué puedo hacer?


In [18]:
# MODIFICACIONES PARA ACELERAR EL PROCESO
# Tiempo previo: 26 min 43 seg
# Tiempo previo2: 24 min 26 seg
# Tiempo Script: 13.08 seg

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from neo4j import GraphDatabase

# Configuración de la conexión a la base de datos Neo4j

# Modelo de embedding
model_name = "BAAI/bge-large-en"

def calculate_similarity(tx, input_query):
    # Obtener los embeddings del query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    result = tx.run("""
    MATCH (n)
    RETURN ID(n) AS node_id, n.NEWembedding AS node_embedding
    """)

    queries_to_update = []

    for record in result:
        node_id = record["node_id"]
        node_embedding = record["node_embedding"]

        # Calcular la similitud del coseno entre el query y el nodo
        similarity_score = cosine_similarity([query_embedding], [node_embedding])[0][0]

        # Preparar la consulta para actualizar el nodo
        queries_to_update.append({
            "node_id": node_id,
            "score": similarity_score
        })

    # Actualizar todos los nodos en una sola transacción
    for query_data in queries_to_update:
        tx.run("""
        MATCH (n)
        WHERE ID(n) = $node_id
        SET n.similaridadquery = $score
        """, node_id=query_data["node_id"], score=query_data["score"])

# Query que deseas comparar con las propiedades de los nodos
query = "Me interesaria saber si el dolor de pecho es un sintoma de cancer de mama"

# Configuración de los embeddings
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
with GraphDatabase.driver(url, auth=(username, password)) as driver:
    embeddings = SentenceTransformer(model_name, **model_kwargs)

    with driver.session() as session:
        session.execute_write(calculate_similarity, query)

print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Me interesaria saber si el dolor de pecho es un sintoma de cancer de mama


In [19]:
# Funcion para calcular el Listado de nodos mas cercanos

def calculate_top_similarity_nodes(tx, input_query):
    # Obtener los embeddings del query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    result = tx.run("""
    MATCH (n)
    RETURN n.node_id AS node_id, n.NEWembedding AS node_embedding, n.similaridadquery AS similarity_score, n.string_es AS node_text
    ORDER BY similarity_score DESC
    LIMIT 5
    """)

    top_nodes = []

    for record in result:
        node_id = record["node_id"]
        node_embedding = record["node_embedding"]
        similarity_score = record["similarity_score"]
        textoNodo = record["node_text"]

        top_nodes.append({
            "node_id": node_id,
            "node_embedding": node_embedding,
            "similarity_score": similarity_score,
            "texto": textoNodo
        })

    return top_nodes

In [20]:
from neo4j import GraphDatabase

print('Pregunta:', query, '\n')

with GraphDatabase.driver(url, auth=(username, password)) as driver:
    with driver.session() as session:
        top_nodes = session.execute_write(calculate_top_similarity_nodes, query)

# Imprimir los 5 nodos con los valores de similitud más altos
for node in top_nodes:
    print(f"Node ID: {node['node_id']}")
    print(f"Similarity Score: {node['similarity_score']}")
    print(f"Texto: {node['texto']}")
    # Puedes acceder al embedding del nodo si es necesario: node['node_embedding']
    print("-------------------------")

Pregunta: Me interesaria saber si el dolor de pecho es un sintoma de cancer de mama 

Node ID: c243
Similarity Score: 0.9164760939052128
Texto: Que me duela el pecho, que tenga grietas o mastitis
-------------------------
Node ID: c2577
Similarity Score: 0.9136390541353228
Texto: Buscar las causas del dolor en el pecho
-------------------------
Node ID: r160
Similarity Score: 0.9120137977021077
Texto: 
 La piel del pecho ha sido creada para que el bebé mame, por tanto, cuando aparecen grietas o dolor, la causa no suele ser que la madre tenga la piel delicada y sí la mala postura al dar el pecho, un mal agarre o un frenillo lingual corto. 
 Si tienes grietas o dolor en el pecho, consulta con tu comadrona y asiste a tu grupo de apoyo a la lactancia más cercano para que puedan ayudarte. El dolor no debería ser normal durante la lactancia y hay que atajarlo cuánto antes.
-------------------------
Node ID: c2421
Similarity Score: 0.9087024586385513
Texto: Mama todo lo que quiere de un 

In [21]:
from neo4j import GraphDatabase

class GraphPathFinder:
    def __init__(self, url, user, password):
        self.driver = GraphDatabase.driver(url, auth=(user, password))

    def close(self):
        self.driver.close()

    # Funcion busqueda Path con profundidad maxima de caminos definida 
    def find_paths(self, start_node_id, max_depth, external_embedding):
        with self.driver.session() as session:
            query = """
            MATCH path = (n {node_id: $start_node_id})-[*1..%s]-(end)
            WHERE all(node in nodes(path) WHERE node.similaridadquery >= 0.7)
            RETURN [node in nodes(path) | [node.node_id, node.similaridadquery, node.string_es]] AS node_info
            ORDER BY reduce(s = 0, n IN nodes(path) | s + n.similaridadquery) DESC
            """ % max_depth
            result = session.run(query, start_node_id=start_node_id)
            return [record["node_info"] for record in result]

# Definir las variables de conexión
    # Parametros definidos previamente

finder = GraphPathFinder(url, username, password)
# Parametros que define la profundidad maxima de los caminos donde buscaremos.
Profund_Maxima_Nodos = 10

try:
    start_node_id = "r160"
    external_embedding = "tu_embedding_externo"  # Actualmente no se usa en la consulta
    paths = finder.find_paths(start_node_id, Profund_Maxima_Nodos, external_embedding)

    # Calcular la media de similaridadquery y almacenar junto con el camino
    path_with_mean = []
    for path in paths:
        if path:
            mean_similarity = sum(node[1] for node in path if node[1] is not None) / len(path)
            path_with_mean.append((path, mean_similarity))

    # Ordenar los caminos por la media de similaridadquery de mayor a menor
    path_with_mean.sort(key=lambda x: x[1], reverse=True)

    # Imprimir los caminos ordenados y sus medias
    for path, mean in path_with_mean:
        path_info = [(node[0], node[1]) for node in path]  # Solo node_id y similaridadquery
        print(f"Camino: {path_info}, Media de Similaridad: {mean}")

    # Camino con la mayor media
    if path_with_mean:
        max_mean_path, max_mean = path_with_mean[0]
        max_mean_path_info = [(node[0], node[1]) for node in max_mean_path]
        max_mean_string_es_values = [node[2] for node in max_mean_path if node[2] is not None]
        print(f"\nCamino con la mayor media de Similaridad: {max_mean_path_info}, Media: {max_mean}")
        print(f"Valores de string_es en el camino óptimo: {max_mean_string_es_values}")
        # Encontrar el nodo con la máxima similaridad en el camino óptimo
        max_similarity_node = max(max_mean_path, key=lambda x: x[1])
        max_similarity_node_info = (max_similarity_node[0], max_similarity_node[1], max_similarity_node[2])
        print(f"Información del nodo con máxima similitud en el camino óptimo: {max_similarity_node_info}")
    else:
        print("No hay caminos o todos los caminos tienen nodos sin similaridadquery")
    print("Pregunta:", query)

finally:
    finder.close()


Camino: [('r160', 0.9120137977021077), ('c2569', 0.8590621399039045)], Media de Similaridad: 0.8855379688030061
Camino: [('r160', 0.9120137977021077), ('c2569', 0.8590621399039045), ('q47', 0.8101116032930865), ('c2568', 0.8731435137933858), ('r943', 0.8791350953547927), ('c1187', 0.8814279926744293)], Media de Similaridad: 0.8691490237869511
Camino: [('r160', 0.9120137977021077), ('c2569', 0.8590621399039045), ('q47', 0.8101116032930865), ('c2568', 0.8731435137933858), ('r943', 0.8791350953547927), ('c1187', 0.8814279926744293), ('q416', 0.8151803719172819), ('c1188', 0.8732522378178817), ('q27', 0.8699050635737605), ('c2577', 0.9136390541353228)], Media de Similaridad: 0.8686870870165955
Camino: [('r160', 0.9120137977021077), ('c2569', 0.8590621399039045), ('q47', 0.8101116032930865), ('c2568', 0.8731435137933858), ('r943', 0.8791350953547927), ('c1187', 0.8814279926744293), ('q416', 0.8151803719172819), ('c1188', 0.8732522378178817), ('q27', 0.8699050635737605), ('t7', 0.90498836762

In [81]:
class GraphPathFinder:
    def __init__(self, url, user, password):
        self.driver = GraphDatabase.driver(url, auth=(user, password))

    def close(self):
        self.driver.close()

    def find_paths(self, start_node_id, max_depth, external_embedding):
        with self.driver.session() as session:
            query = """
            MATCH path = (n {node_id: $start_node_id})-[*1..%s]-(end)
            WHERE all(node in nodes(path) WHERE node.similaridadquery >= 0.7)
            RETURN [node in nodes(path) | [node.node_id, node.similaridadquery, node.string_es]] AS node_info
            ORDER BY reduce(s = 0, n IN nodes(path) | s + n.similaridadquery) DESC
            """ % max_depth
            result = session.run(query, start_node_id=start_node_id)
            return [record["node_info"] for record in result]
    def get_all_theme_strings(self):
        with self.driver.session() as session:
            query = """
            MATCH (theme:Theme)
            RETURN collect(theme.string_es) AS theme_strings
            """
            result = session.run(query)
            return [record["theme_strings"] for record in result][0]
    def get_node_ids_by_theme_string(self, theme_string):
        with self.driver.session() as session:
            query = """
            MATCH (node:Theme {string_es: $theme_string})
            RETURN collect(node.node_id) AS node_ids
            """
            result = session.run(query, theme_string=theme_string)
            return [record["node_ids"] for record in result][0]

In [64]:
def procesar_nodos(finder, start_node_ids, Profund_Maxima_Nodos, external_embedding=None):
    resultados = []
    for start_node_id in start_node_ids:
        paths = finder.find_paths(start_node_id, Profund_Maxima_Nodos, external_embedding)

        # Calcula la media de similaridad y almacena junto con el camino
        path_with_mean = []
        for path in paths:
            if path:
                mean_similarity = sum(node[1] for node in path if node[1] is not None) / len(path)
                path_with_mean.append((path, mean_similarity))

        # Ordena los caminos por la media de similaridad de mayor a menor
        path_with_mean.sort(key=lambda x: x[1], reverse=True)

        # Guarda la información de los caminos y la máxima media
        if path_with_mean:
            max_mean_path, max_mean = path_with_mean[0]
            max_mean_path_info = [(node[0], node[1]) for node in max_mean_path]
            max_mean_string_es_values = [node[2] for node in max_mean_path if node[2] is not None]
            max_similarity_node = max(max_mean_path, key=lambda x: x[1])
            max_similarity_node_info = (max_similarity_node[0], max_similarity_node[1], max_similarity_node[2])

            resultados.append({
                'start_node_id': start_node_id,
                'max_mean_path_info': max_mean_path_info,
                'max_mean': max_mean,
                'max_mean_string_es_values': max_mean_string_es_values,
                'max_similarity_node_info': max_similarity_node_info
            })

    return resultados

In [82]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)

# Lista de identificadores de nodo iniciales
start_node_ids = ["r160", "t29"]  # Lista de nodos iniciales

# Profundidad máxima de búsqueda
Profund_Maxima_Nodos = 5

# Llamada a la función procesar_nodos
resultados = procesar_nodos(finder, start_node_ids, Profund_Maxima_Nodos)

# Imprimir resultados
for resultado in resultados:
    print(resultado)

# Cerrar la conexión
finder.close()

{'start_node_id': 'r160', 'max_mean_path_info': [('r160', 0.9120137977021077), ('c2569', 0.8590621399039045)], 'max_mean': 0.8855379688030061, 'max_mean_string_es_values': ['\n La piel del pecho ha sido creada para que el bebé mame, por tanto, cuando aparecen grietas o dolor, la causa no suele ser que la madre tenga la piel delicada y sí la mala postura al dar el pecho, un mal agarre o un frenillo lingual corto. \n Si tienes grietas o dolor en el pecho, consulta con tu comadrona y asiste a tu grupo de apoyo a la lactancia más cercano para que puedan ayudarte. El dolor no debería ser normal durante la lactancia y hay que atajarlo cuánto antes.', 'Me dicen que mi piel es muy delicada'], 'max_similarity_node_info': ('r160', 0.9120137977021077, '\n La piel del pecho ha sido creada para que el bebé mame, por tanto, cuando aparecen grietas o dolor, la causa no suele ser que la madre tenga la piel delicada y sí la mala postura al dar el pecho, un mal agarre o un frenillo lingual corto.

In [83]:
def procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths, external_embedding=None):
    resultados = []
    for start_node_id in start_node_ids:
        paths = finder.find_paths(start_node_id, Profund_Maxima_Nodos, external_embedding)

        # Calcula la media de similaridad y almacena junto con el camino
        path_with_mean = []
        for path in paths:
            if path:
                mean_similarity = sum(node[1] for node in path if node[1] is not None) / len(path)
                path_with_mean.append((path, mean_similarity))

        # Ordena los caminos por la media de similaridad de mayor a menor
        path_with_mean.sort(key=lambda x: x[1], reverse=True)

        # Guarda la información de los n caminos con mayor media
        top_paths = path_with_mean[:n_top_paths]
        for path, mean in top_paths:
            path_info = [(node[0], node[1]) for node in path]
            string_es_values = [node[2] for node in path if node[2] is not None]
            max_similarity_node = max(path, key=lambda x: x[1])
            max_similarity_node_info = (max_similarity_node[0], max_similarity_node[1], max_similarity_node[2])

            resultados.append({
                'start_node_id': start_node_id,
                'path_info': path_info,
                'mean_similarity': mean,
                'string_es_values': string_es_values,
                'max_similarity_node_info': max_similarity_node_info
            })

    return resultados

In [84]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)

# Lista de identificadores de nodo iniciales
start_node_ids = ["r160", "t29"]  # Lista de nodos iniciales

# Profundidad máxima de búsqueda
Profund_Maxima_Nodos = 5

# Número de caminos con mayor media de similaridad a mostrar
n_top_paths = 20

# Llamada a la función procesar_nodos
resultados = procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths)

# Imprimir resultados
for resultado in resultados:
    print(resultado)

# Cerrar la conexión
finder.close()

{'start_node_id': 'r160', 'path_info': [('r160', 0.9120137977021077), ('c2569', 0.8590621399039045)], 'mean_similarity': 0.8855379688030061, 'string_es_values': ['\n La piel del pecho ha sido creada para que el bebé mame, por tanto, cuando aparecen grietas o dolor, la causa no suele ser que la madre tenga la piel delicada y sí la mala postura al dar el pecho, un mal agarre o un frenillo lingual corto. \n Si tienes grietas o dolor en el pecho, consulta con tu comadrona y asiste a tu grupo de apoyo a la lactancia más cercano para que puedan ayudarte. El dolor no debería ser normal durante la lactancia y hay que atajarlo cuánto antes.', 'Me dicen que mi piel es muy delicada'], 'max_similarity_node_info': ('r160', 0.9120137977021077, '\n La piel del pecho ha sido creada para que el bebé mame, por tanto, cuando aparecen grietas o dolor, la causa no suele ser que la madre tenga la piel delicada y sí la mala postura al dar el pecho, un mal agarre o un frenillo lingual corto. \n Si tien

In [85]:
def mostrar_resultado(resultado, mostrar):
    ''' Imprime el resultado de la búsqueda de caminos en la base de datos Neo4j
    limitando el número de nodos del camino mostrados
    y valores de string_es a mostrar.
    el valor limite dado por: 'mostrar'
    '''
    print("ID del Nodo Inicial:", resultado['start_node_id'])

    print("\nInformación del Camino (mostrando hasta {} nodos):".format(mostrar))
    for node in resultado['path_info'][:mostrar]:
        print(f"  Nodo ID: {node[0]}, Similaridad: {node[1]}")

    print("\nMedia de Similaridad:", resultado['mean_similarity'])

    print("\nValores de string_es (mostrando hasta {} valores):".format(mostrar))
    for value in resultado['string_es_values'][:mostrar]:
        print(f"  {value}")

    print("\nInformación del Nodo con Máxima Similaridad:")
    max_node_info = resultado['max_similarity_node_info']
    print(f"  Nodo ID: {max_node_info[0]}, Similaridad: {max_node_info[1]}")
    print(f"  string_es: {max_node_info[2]}")

In [86]:
print("Pregunta:", query, "\n")
for s in range(0, 2):
    print("Resultado:", s)
    mostrar_resultado(resultados[s], 8)

Pregunta: Me interesaria saber si el dolor de pecho es un sintoma de cancer de mama 

Resultado: 0
ID del Nodo Inicial: r160

Información del Camino (mostrando hasta 8 nodos):
  Nodo ID: r160, Similaridad: 0.9120137977021077
  Nodo ID: c2569, Similaridad: 0.8590621399039045

Media de Similaridad: 0.8855379688030061

Valores de string_es (mostrando hasta 8 valores):
  
 La piel del pecho ha sido creada para que el bebé mame, por tanto, cuando aparecen grietas o dolor, la causa no suele ser que la madre tenga la piel delicada y sí la mala postura al dar el pecho, un mal agarre o un frenillo lingual corto. 
 Si tienes grietas o dolor en el pecho, consulta con tu comadrona y asiste a tu grupo de apoyo a la lactancia más cercano para que puedan ayudarte. El dolor no debería ser normal durante la lactancia y hay que atajarlo cuánto antes.
  Me dicen que mi piel es muy delicada

Información del Nodo con Máxima Similaridad:
  Nodo ID: r160, Similaridad: 0.9120137977021077
  string_es: 
 L

Coger las preguntas de las usuarias

In [87]:

FILEDATOS = '/media/ubuntu/Linux3/02_TFM/TEST_BIBLIOTECA/MODELOS/a.xlsx'

import pandas as pd
import numpy as np
df = pd.read_excel(FILEDATOS)

In [59]:
mensajes = df['message'].to_list()
mensajes = list(set(mensajes))

Analisis de Tematicas de los mensajes

In [1]:
# Bibliotecas Necesarias Para Funcionamiento Modelos en formato GGUF

!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]

In [48]:
# Bibliotecas LangChain

from langchain.llms import CTransformers

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [2]:
# Listado de posibles modelos a utilizar para pruebas:

# Ref: HuggingFace Models

####################################################
# model_id = "TheBloke/Llama-2-7B-GGML"
# model_id = "TheBloke/Llama-2-7B-chat-GGML"
# model_id = "TheBloke/Llama-2-13B-GGML"
# model_id = "TheBloke/Llama-2-13B-chat-GGML"
####################################################

# Modelos:
# 4.1.1- Mistral7b
modelM = "TheBloke/OpenHermes-2.5-Mistral-7B-GGUF"
model_fileM = "openhermes-2.5-mistral-7b.Q3_K_M.gguf"

# 4.1.2.- Llama2
modelChat = 'TheBloke/Llama-2-7B-Chat-GGUF'
model_fileChat="llama-2-7b-chat.Q3_K_M.gguf"

modelD='TheBloke/Llama-2-7B-GGUF'
model_fileD="llama-2-7b.Q3_K_M.gguf"

modelChatLL = 'TheBloke/Llama-2-13B-chat-GGUF'
model_fileLL = 'llama-2-13b-chat.Q3_K_M.gguf'

In [3]:
# Parametros Modelos
config = {
          'max_new_tokens': 512,
          'repetition_penalty': 1.1,
          'temperature': 0,
          'context_length': 1024,
          'stream': False
          }

| Parameter            | Type      | Description                                                     | Default   |
|----------------------|-----------|-----------------------------------------------------------------|-----------|
| top_k                | int       | The top-k value to use for sampling.                            | 40        |
| top_p                | float     | The top-p value to use for sampling.                            | 0.95      |
| temperature          | float     | The temperature to use for sampling.                            | 0.8       |
| repetition_penalty   | float     | The repetition penalty to use for sampling.                     | 1.1       |
| last_n_tokens        | int       | The number of last tokens to use for repetition penalty.        | 64        |
| seed                 | int       | The seed value to use for sampling tokens.                      | -1        |
| max_new_tokens       | int       | The maximum number of new tokens to generate.                   | 256       |
| stop                 | List[str] | A list of sequences to stop generation when encountered.        | None      |
| stream               | bool      | Whether to stream the generated text.                           | False     |
| reset                | bool      | Whether to reset the model state before generating text.        | True      |
| batch_size           | int       | The batch size to use for evaluating tokens in a single prompt. | 8         |
| threads              | int       | The number of threads to use for evaluating tokens.             | -1        |
| context_length       | int       | The maximum context length to use.                              | -1        |
| gpu_layers           | int       | The number of layers to run on GPU.                             | 0         |

In [37]:
## Caso de usar .env file

import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

# Accede a las variables de entorno en tu código
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Ahora, puedes usar las variables en tu script
#print(f'OPENAI_API_KEY: {OPENAI_API_KEY}')

In [40]:
# Carga Modelo Open AI

#modelOpenAI = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

modelOpenAI = ChatOpenAI(temperature=0)

In [55]:
# Ejemplo de Prompts condicionales

#Ref: https://python.langchain.com/docs/guides/local_llms
# Mistral --> https://smith.langchain.com/hub/rlm/rag-prompt-mistral
# Llama --> https://smith.langchain.com/hub/rlm/rag-prompt-llama


from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector

from typing import List, Tuple


DEFAULT_MISTRAL_SEARCH_PROMPT = PromptTemplate(
    input_variables=["texto", "categorias"],
    template="""<s> [INST] You are an expert in breastfeeding.\
If you don't know the answer, just say that you don't know. \
The output must be only the category. [/INST] </s> \
[INST] Question: classify the text: {texto}. \
Context: The categories for classify are: {categorias}. \
Answer: [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["texto", "categorias"],
    template=""""Clasifica el siguiente texto en una de las siguientes categorías {categorias}: \n Texto: {texto}\nCategoría: """,
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[
        (lambda model: (model == llm), DEFAULT_MISTRAL_SEARCH_PROMPT)
    ],
)


prompt2 = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt3 = QUESTION_PROMPT_SELECTOR.get_prompt(modelOpenAI)



print('Mistral7:\n', prompt2)

print('OpenAI:\n', prompt3)

Mistral7:
 input_variables=['categorias', 'texto'] template="<s> [INST] You are an expert in breastfeeding.If you don't know the answer, just say that you don't know. The output must be only the category. [/INST] </s> [INST] Question: classify the text: {texto}. Context: The categories for classify are: {categorias}. Answer: [/INST]"
OpenAI:
 input_variables=['categorias', 'texto'] template='"Clasifica el siguiente texto en una de las siguientes categorías {categorias}: \n Texto: {texto}\nCategoría: '


In [56]:
# Caso Mistral y Llama2

chainMistral = prompt2 | llm | StrOutputParser()

chainOpenAI = prompt3 | modelOpenAI | StrOutputParser()

In [58]:
def clasificar_LLMs(texto, categorias, chain):
    '''Función de clasificación
    Entrada:
        texto -> Conversación Usuaria
        categorias --> Lista de categorias donde se ha de clasificar
        chain --> Cadena de LangChain
    respuesta:
        Respuesta del modelo
    '''
    response = chain.invoke({"categorias": categorias, "texto": texto})
    return response

In [71]:
entrada = mensajes[0]

In [88]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)
categorias = finder.get_all_theme_strings()
categorias = list(set(categorias))
print(categorias)
print(len(categorias))

['MI BEBÉ MAMA MUCHO', 'SOBRE MI SALUD', 'LACTANCIA MIXTA', 'COSAS QUE HACE MI BEBÉ', '¿PUEDO...?', 'CONCEPTOS BÁSICOS', 'Lactancia artificial', 'MIS SENTIMIENTOS', 'Visitas y talleres', 'BULTOS EN EL PECHO', 'RELACTAR', 'INDUCCIÓN', 'GRIETAS Y HERIDAS', 'ESTOY ENFERMA', 'PRODUCTOS PARA LA LACTANCIA', 'ME HE QUEDADO EMBARAZADA', 'BACHES Y CRISIS', 'Baby Led Weaning', 'Guía de regalos', 'PAÑALES SUCIOS', '¿TENGO POCA LECHE?', 'PECHO: FORMA Y TAMAÑO', 'Mi embarazo por etapas', 'EMBARAZO: SEGUIR LACTANDO', 'SUEÑO Y LACTANCIA', '¡QUIERO SER DONANTE!', 'CÓMO PRODUCIR MÁS LECHE', 'EL CAMINO DE LA LECHE', 'ALIMENTOS SÓLIDOS', 'SALUD DE MI BEBÉ', 'EL DESTETE POR ETAPAS', 'Durante mi embarazo', 'MI BEBÉ NO GANA PESO', 'GUÍA DEL DOLOR AL AMAMANTAR', 'Punto Violeta', 'LA VUELTA AL TRABAJO', 'DEJAR LA FÓRMULA', 'PRUEBAS DIAGNÓSTICAS', 'PORTEO', 'Cómo gestionar mi mastitis', '¿QUÉ HAGO SI...?', 'LACTANCIA POR ETAPAS', 'EXTRACCIÓN DE LECHE', 'Coronavirus', 'DOLOR EN EL PECHO', 'EN

In [76]:
salidaMistral = clasificar_LLMs(entrada, categorias, chainMistral)

Number of tokens (881) exceeded maximum context length (512).
Number of tokens (882) exceeded maximum context length (512).
Number of tokens (883) exceeded maximum context length (512).
Number of tokens (884) exceeded maximum context length (512).
Number of tokens (885) exceeded maximum context length (512).
Number of tokens (886) exceeded maximum context length (512).
Number of tokens (887) exceeded maximum context length (512).
Number of tokens (888) exceeded maximum context length (512).
Number of tokens (889) exceeded maximum context length (512).
Number of tokens (890) exceeded maximum context length (512).
Number of tokens (891) exceeded maximum context length (512).
Number of tokens (892) exceeded maximum context length (512).
Number of tokens (893) exceeded maximum context length (512).
Number of tokens (894) exceeded maximum context length (512).
Number of tokens (895) exceeded maximum context length (512).
Number of tokens (896) exceeded maximum context length (512).
Number o

In [77]:
salidaMistral

" </s> </s > ['SOBATCH=MI BEGive] </s> ] </s> </s> </s> </s] <s] I really enjoyed ] LACT: SOBULT] </s> </s]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n"

In [78]:
salidaOpen= clasificar_LLMs(entrada, categorias, chainOpenAI)

In [80]:
print(entrada)
print(salidaOpen)


Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está relacionado. He leído algo sobre hongos en el pecho, pero espero que no sea el caso. Crees que podría ser eso? Un saludo y muchísimas gracias!
GRIETAS Y HERIDAS


In [90]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)
ids = finder.get_node_ids_by_theme_string(salidaOpen)
ids = list(set(ids))
print(ids)
print(len(ids))

['t9']
1


In [183]:
for mensaje in mensajes[:1]:
    print("MENSAJE:", mensaje)
    print("-------------------------")
    # Query que deseas comparar con las propiedades de los nodos
    query = mensaje
    # Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
    with GraphDatabase.driver(url, auth=(username, password)) as driver:
        embeddings = SentenceTransformer(model_name, **model_kwargs)

        with driver.session() as session:
            session.execute_write(calculate_similarity, query)

    print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

    # Identificación de los temas de la pregunta
    salidaOpen= clasificar_LLMs(mensaje, categorias, chainOpenAI)

    # Creación de la instancia de GraphPathFinder
    finder = GraphPathFinder(url, username, password)
    ids = finder.get_node_ids_by_theme_string(salidaOpen)

    # Lista de identificadores de nodo iniciales
    start_node_ids = ids  # Lista de nodos iniciales

    # Profundidad máxima de búsqueda
    Profund_Maxima_Nodos = 7

    # Número de caminos con mayor media de similaridad a mostrar
    n_top_paths = 5

    # Llamada a la función procesar_nodos
    resultados = procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths)

    mostrar_resultado(resultados[0], Profund_Maxima_Nodos)
    # Imprimir resultados
    #for resultado in resultados:
    #    print(resultado)

    # Cerrar la conexión
    finder.close()

    

MENSAJE: Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está relacionado. He leído algo sobre hongos en el pecho, pero espero que no sea el caso. Crees que podría ser eso? Un saludo y muchísimas gracias!
-------------------------
Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está r

RESUMEN DE RESULTADOS

In [184]:
# PREGUNTAS DE PRUEBA
print("Pregunta:", query, "\n")


Pregunta: Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está relacionado. He leído algo sobre hongos en el pecho, pero espero que no sea el caso. Crees que podría ser eso? Un saludo y muchísimas gracias! 



In [185]:
# Tematica identificada
print('Tematica:', salidaOpen)

Tematica: GRIETAS Y HERIDAS


In [186]:
# Nodo de tematica identificada
print('Nodo:', ids)

Nodo: ['t9']


In [187]:
# Nodos con mayor similaridad
for t in range(0, n_top_paths):
    tuplas = resultados[t]['path_info']
    # Extracción del primer elemento de cada tupla
    primeros_elementos= [tupla[0] for tupla in tuplas]
    # Mostrar la nueva lista
    print('Nodos camino identificado:', primeros_elementos)

Nodos camino identificado: ['t9', 'q124', 'c2884', 'r2114']
Nodos camino identificado: ['t9', 'q124', 'c489', 'q121', 'c3381', 'r2538']
Nodos camino identificado: ['t9', 'q124', 'c489', 'q121', 'c491', 'r623', 'c3214']
Nodos camino identificado: ['t9', 'q124', 'c489', 'q121', 'c2834', 'r554', 'c445', 'q106']
Nodos camino identificado: ['t9', 'q124', 'c489', 'q121', 'c2834', 'r554', 'c445']


In [188]:
# CADENA TEXTOS
print('Numero de cadenas:', n_top_paths)
# Iterar sobre diferentes cadenas identificadas
for n in range(0, n_top_paths):
    print('Cadena: ',n, "Similaridad Media: ",resultados[n]['mean_similarity'], '\n')
    print("Pregunta: ", query)
    for t, texto in enumerate(resultados[n]['string_es_values']):
        tuplas = resultados[n]['path_info']
        # Extracción del primer elemento de cada tupla
        primeros_elementos= [tupla[0] for tupla in tuplas]
        print(t,': (Nodo:', primeros_elementos[t],')', texto)
    print('-------------------------')


Numero de cadenas: 5
Cadena:  0 Similaridad Media:  0.8514941560434868 

Pregunta:  Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está relacionado. He leído algo sobre hongos en el pecho, pero espero que no sea el caso. Crees que podría ser eso? Un saludo y muchísimas gracias!
0 : (Nodo: t9 ) GRIETAS Y HERIDAS
1 : (Nodo: q124 ) ¿En qué te podemos ayudar?
2 : (Nodo: c2884 ) Consultar las partes del pezón y la areola
3 : (Nodo: r2114 ) 
 Es indispensable que conozcas las diferentes partes del pezón y la areola para poder indicar en qué zona está la grieta. La siguiente imágen te ayudará a poder indentificarlas y poder hacer mejor la consulta. 

------------------------

In [189]:
# Extracción del segundo elemento de cada tupla
segundos_elementos= [tupla[1] for tupla in tuplas]
print('Similaridad:', segundos_elementos)   

Similaridad: [0.820924178608837, 0.8289856877010512, 0.8558724270849432, 0.8114395779638116, 0.8784509737271468, 0.8914362529153453, 0.8488960530752125]


In [107]:
# Similaridad media
print('Similaridad media:', resultados[0]['mean_similarity'])

Similaridad media: 0.8400905034196252


In [108]:
# Valores de string_es
print('Texto Nodos:', resultados[0]['string_es_values'])

Valores de string_es: ['¿PUEDO...?', '¿Qué tema quieres consultar?', 'Sobre la leche', 'Elige un tema:', 'Si tengo mastitis o infección, ¿puedo seguir amamantando?', '\n En el caso de que sufras mastitis u obstrucción, es recomendable que no dejes de amamantar al bebé hasta que no te hayas repuesto . \n Amamantar con frecuencia te ayudará a drenar el pecho y evitar que empeore la situación. Recuerda que no existe ningún riesgo para el bebé y puede seguir mamando. \n Durante la mastitis, el sabor de la leche se suele modificar y temporalmente está más salada. Si el bebé rechaza el pecho por esta razón, será imprescindible que te saques leche a mano o con el sacaleches para evitar empeorar el cuadro.']


In [190]:
import json

# CADENA TEXTOS
n_top_paths = 3  # Cambia esto según tu necesidad

# Crear una lista para almacenar los datos de salida
output_data = []

# Iterar sobre diferentes cadenas identificadas
for n in range(0, n_top_paths):
    cadena_data = {
        'Cadena': n,
        'Similaridad_Media': resultados[n]['mean_similarity'],
        'Pregunta': query,
        'Nodos_Textos': []
    }

    for t, texto in enumerate(resultados[n]['string_es_values']):
        tuplas = resultados[n]['path_info']
        primeros_elementos = [tupla[0] for tupla in tuplas]
        nodo_texto = {
            'Nodo': primeros_elementos[t],
            'Texto': texto
        }
        cadena_data['Nodos_Textos'].append(nodo_texto)

    output_data.append(cadena_data)

# Serializar los datos de salida como JSON
json_output = json.dumps(output_data, ensure_ascii=False, indent=4)

# Imprimir la salida JSON
print(json_output)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.8514941560434868,
        "Pregunta": "Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está relacionado. He leído algo sobre hongos en el pecho, pero espero que no sea el caso. Crees que podría ser eso? Un saludo y muchísimas gracias!",
        "Nodos_Textos": [
            {
                "Nodo": "t9",
                "Texto": "GRIETAS Y HERIDAS"
            },
            {
                "Nodo": "q124",
                "Texto": "¿En qué te podemos ayudar?"
            },
            {
                "Nodo": "c2884",
                "Texto": "Consultar las partes del pezón y la areola"
            },
        

In [191]:
# Cargar el JSON
data = json.loads(json_output)

# Imprimir el JSON formateado en pantalla
formatted_json = json.dumps(data, ensure_ascii=False, indent=4)
print(formatted_json)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.8514941560434868,
        "Pregunta": "Gracias por una respuesta tan rápida!! Pues al tener la niña en el pecho siento como una especie de escozor. Lo he asociado a que come con más frecuencia y los pechos no descansan. Pero lo que más me preocupa es el hecho de que a veces se enganche unos segundos, se suelte y llore. A veces también la niña tira del pezon y succiona con mucha fuerza. No sé si todo está relacionado. He leído algo sobre hongos en el pecho, pero espero que no sea el caso. Crees que podría ser eso? Un saludo y muchísimas gracias!",
        "Nodos_Textos": [
            {
                "Nodo": "t9",
                "Texto": "GRIETAS Y HERIDAS"
            },
            {
                "Nodo": "q124",
                "Texto": "¿En qué te podemos ayudar?"
            },
            {
                "Nodo": "c2884",
                "Texto": "Consultar las partes del pezón y la areola"
            },
        

In [25]:
import lmql

In [33]:
import nest_asyncio
nest_asyncio.apply()

In [26]:
lmql.model("local:llama.cpp:openhermes-2.5-mistral-7b.Q3_K_M.gguf")




/media/ubuntu/Linux3/CHAT_BOT_FINANCIERO/venv/lib/python3.11/site-packages/lmql/models/lmtp/lmtp_dcinprocess.py:36: UserWarning: By default LMQL uses the 'huggyllama/llama-7b' tokenizer for all llama.cpp models. To change this, set the 'tokenizer' argument of your lmql.model(...) object.
  warnings.warn("By default LMQL uses the '{}' tokenizer for all llama.cpp models. To change this, set the 'tokenizer' argument of your lmql.model(...) object.".format("huggyllama/llama-7b", UserWarning))


lmql.LLM(local:llama.cpp:openhermes-2.5-mistral-7b.Q3_K_M.gguf, )

In [40]:

@lmql.query(model="openhermes-2.5-mistral-7b.Q3_K_M.gguf")
def chain_of_thought(question):
    '''lmql
    # Q&A prompt template
    "Q: {question}\n"
    "A: Let's think step by step.\n"
    "[REASONING]"
    "Thus, the answer is:[ANSWER]."

    # return just the ANSWER to the caller
    return ANSWER
    '''


In [ ]:

load_vector_store = Reply_embedding

retriever = load_vector_store.as_retriever(search_kwargs={"k":1})

def get_response():
    query = request.form.get('query')
    # Your logic to handle the query
    chain_type_kwargs = {"prompt": prompt},
    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=retriever
    )

escribe un script pitón para usando una divertir diversión llamada sera la que sera la que sea el computo de la propiedad de la relacion

funcionExterna que toma dos valores de similitud y devuelve un valor calculado. También asumiremos que estás utilizando el driver oficial de Neo4j para Python.

In [ ]:
def funcionExterna(similitud1, similitud2):
    # Ejemplo de función, que podría ser cualquier cálculo
    return (similitud1 + similitud2) / 2


In [ ]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
user = "neo4j"
password = "password"

# Conexión a Neo4j
driver = GraphDatabase.driver(uri, auth=(user, password))

def actualizar_relaciones():
    with driver.session() as session:
        # Obtener todas las relaciones y sus nodos adyacentes
        relaciones = session.run("MATCH (n1)-[r]->(n2) RETURN id(r) AS idRelacion, n1.similarity AS sim1, n2.similarity AS sim2")
        
        for relacion in relaciones:
            # Calcular la nueva propiedad usando la función externa
            nueva_propiedad = funcionExterna(relacion["sim1"], relacion["sim2"])
            
            # Actualizar la propiedad de la relación en Neo4j
            session.run("MATCH ()-[r]->() WHERE id(r) = $idRelacion SET r.similarityAverage = $nuevaPropiedad", 
                        idRelacion=relacion["idRelacion"], nuevaPropiedad=nueva_propiedad)

# Ejecutar la función de actualización
actualizar_relaciones()

# Cerrar la conexión
driver.close()

Tenemos un grafo en que cada nodo tiene una propiedad que es la medida de la similitud entre un vector de incrustar del nodo y un incrustación externo. Dado un nodo inicial, en encontrar de entre los caminos que salen de ese nodo el que se trata de una mejor similitud en cada nodo con el vector externo. Utilizo neo4j. Escribe un guión en python para obtener una lista de caminos ordenados de ayor similitus a menor.

In [ ]:
from neo4j import GraphDatabase

class GraphPathFinder:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def find_paths(self, start_node_id, external_embedding):
        with self.driver.session() as session:
            query = """
            MATCH path = (start:Node {id: $start_node_id})-[*]->(end)
            WHERE all(node in nodes(path) WHERE node.embedding_similarity >= threshold)
            RETURN path
            ORDER BY reduce(s = 0, n IN nodes(path) | s + n.embedding_similarity) DESC
            """
            result = session.run(query, start_node_id=start_node_id)
            return [record["path"] for record in result]

# Uso del script
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "your_password"
finder = GraphPathFinder(uri, user, password)

try:
    start_node_id = "your_start_node_id"
    external_embedding = "your_external_embedding_vector"
    paths = finder.find_paths(start_node_id, external_embedding)
    for path in paths:
        print(path)
finally:
    finder.close()

########################################

In [202]:
def listar_archivos(directorio):
    """
    Esta función lista los archivos en el directorio dado.

    :param directorio: Ruta del directorio a listar
    """
    try:
        # Listar todos los archivos y directorios en el directorio dado
        archivos = os.listdir(directorio)
        print(f"Archivos en '{directorio}':")
        
        # Iterar a través de los archivos y directorios
        for archivo in archivos:
            # Ruta completa del archivo
            ruta_completa = os.path.join(directorio, archivo)
            
            # Verificar si es un archivo o un directorio
            if os.path.isfile(ruta_completa):
                print(f"Archivo: {archivo}")
            else:
                print(f"Directorio: {archivo}")
    except Exception as e:
        print(f"Error: {e}")



In [203]:
# Uso del script
directorio = '/media/ubuntu/Linux3/02_TFM/DATOS_APP'
listar_archivos(directorio)

Archivos en '/media/ubuntu/Linux3/02_TFM/DATOS_APP':
Archivo: conversations_no_chitchat.csv
Archivo: conversations.csv


In [205]:
def leer_csv_especifico(directorio, nombre_archivo):
    """
    Esta función lee un archivo .csv específico en el directorio dado.

    :param directorio: Ruta del directorio donde buscar el archivo
    :param nombre_archivo: Nombre del archivo .csv a leer
    :return: DataFrame de pandas con los datos del archivo .csv, o None si el archivo no se encuentra o hay un error
    """
    try:
        ruta_archivo_csv = os.path.join(directorio, nombre_archivo)
        
        # Verificar si el archivo existe
        if os.path.exists(ruta_archivo_csv) and ruta_archivo_csv.endswith('.csv'):
            df = pd.read_csv(ruta_archivo_csv)
            print(f"Archivo CSV leído: {nombre_archivo}")
            return df
        else:
            print(f"No se encontró el archivo '{nombre_archivo}' en el directorio.")
            return None
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return None

In [208]:
# Uso del script
df = leer_csv_especifico(directorio, 'conversations.csv')

Archivo CSV leído: conversations.csv


In [209]:
df.head(5)

,treeName,msg_id,db_user_id,link,message,user_id,expert,tree,time,discard,classified,expert_old,empty,conversation_id
0,Classifying,5e5ea4db4196ca0011c5fdbf,33020,Classifying,Hola! Buenas tardes! Una pregunta: Una mami de...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583261e+09,Classifying,True,1MP_Laura,0,1
1,Classifying,5e5f501d378264000f05ceaf,33020,Classifying,"Vale, le diré. Muchas gracias!",000121b28e5a9eda9f9fecf5,user,Classifying,1.583305e+09,Classifying,True,1MP_Laura,0,1
2,Classifying,5e5f50da9ce47d000e7c2a53,33020,Classifying,Buenos dias! Aprovecho para haceros otra cons...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583305e+09,Classifying,True,1MP_Laura,0,1
3,Classifying,5e5f61f7fdba80000f3b12f9,33020,Classifying,Aun en dosis muy altas? En mi caso por ejemplo...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583309e+09,Classifying,True,1MP_Laura,0,1
4,Classifying,5e5f6227378264000f068fed,33020,Classifying,Y conocéis algún otro fármaco mas? Compatible ...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583309e+09,Classifying,True,1MP_Laura,0,1


In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663814 entries, 0 to 663813
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   treeName         663814 non-null  object 
 1   msg_id           663814 non-null  object 
 2   db_user_id       663814 non-null  int64  
 3   link             663814 non-null  object 
 4   message          663814 non-null  object 
 5   user_id          663814 non-null  object 
 6   expert           663814 non-null  object 
 7   tree             663814 non-null  object 
 8   time             663814 non-null  float64
 9   discard          663814 non-null  object 
 10  classified       81886 non-null   object 
 11  expert_old       663814 non-null  object 
 12  empty            663814 non-null  int64  
 13  conversation_id  663814 non-null  int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 70.9+ MB


In [210]:
# Uso del script
df1 = leer_csv_especifico(directorio, 'conversations_no_chitchat.csv')

Archivo CSV leído: conversations_no_chitchat.csv


In [211]:
df1.head(5)

,treeName,msg_id,db_user_id,link,message,user_id,expert,tree,time,discard,classified,expert_old,empty,conversation_id
0,Classifying,5e5ea4db4196ca0011c5fdbf,33020,Classifying,"Una pregunta: Una mami de cadiz, de la linea d...",000121b28e5a9eda9f9fecf5,user,Classifying,1.583261e+09,Classifying,True,1MP_Laura,0,1
1,Classifying,5e5f501d378264000f05ceaf,33020,Classifying,"Vale, le diré.",000121b28e5a9eda9f9fecf5,user,Classifying,1.583305e+09,Classifying,True,1MP_Laura,0,1
2,Classifying,5e5f50da9ce47d000e7c2a53,33020,Classifying,Aprovecho para haceros otra consulta... Coord...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583305e+09,Classifying,True,1MP_Laura,0,1
3,Classifying,5e5f61f7fdba80000f3b12f9,33020,Classifying,Aun en dosis muy altas? En mi caso por ejemplo...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583309e+09,Classifying,True,1MP_Laura,0,1
4,Classifying,5e5f6227378264000f068fed,33020,Classifying,Y conocéis algún otro fármaco mas? Compatible ...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583309e+09,Classifying,True,1MP_Laura,0,1


In [213]:
df1.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663814 entries, 0 to 663813
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   treeName         663814 non-null  object 
 1   msg_id           663814 non-null  object 
 2   db_user_id       663814 non-null  int64  
 3   link             663814 non-null  object 
 4   message          542001 non-null  object 
 5   user_id          663814 non-null  object 
 6   expert           663814 non-null  object 
 7   tree             663814 non-null  object 
 8   time             663814 non-null  float64
 9   discard          663814 non-null  object 
 10  classified       81886 non-null   object 
 11  expert_old       663814 non-null  object 
 12  empty            663814 non-null  int64  
 13  conversation_id  663814 non-null  int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 70.9+ MB


In [214]:
mensajesProfe = df['message'].to_list()
mensajesProfe = list(set(mensajesProfe))
len(mensajesProfe)

553129

In [216]:
corte = mensajesProfe[20:30]
corte

['Sí, parece que tu bebé tiene algo de retrognatia. Las posiciones verticales te pueden favorecer el agarre',
 'https://media.smooch.io/apps/5a1d38ccc5c9b0004b537431/conversations/1c8409d7471e176043f7eaed/b1mZNCHjWfZkYfi-UjfEduca/image.jpg',
 'Hola Encarna, encantada de saludarte, gracias por tu confianza en LactApp. Felicidades por tu bebé',
 'el resto, nada',
 'Reducimos de 4 a 2 suplementos en una semana y ha perdido 80gr. Por eso hemos vuelto a 3, segun una pediatra experta en lactancia',
 'Hasta ahora mamaba normalmente? Solo se alimenta de pecho?',
 'Llevaba 3 meses utilizándolo',
 '¡Hola!',
 'Hola! Es normal que mi bebé de 6 semanas quiera comer a cada hora en las últimas horas del día? Toma LME. Anoche también lo hizo durante la madrugada. No puedo dormirla entre toma y toma. ',
 'Ha estado con mocos']